In [2]:
import tensorflow as tf

2022-07-11 22:07:30.591644: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-11 22:07:30.837969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-centos8-cascadelake/gcc-9.3.0/julia-1.5.2-3z/lib:/opt/ohpc/pub/libs/hwloc/2.1.0/lib:/opt/ohpc/pub/mpi/libfabric/1.10.1/lib:/opt/ohpc/pub/mpi/ucx-ohpc/1.8.0/lib:/opt/ohpc/pub/mpi/openmpi4-gnu9/4.0.4/lib:/opt/ohpc/pub/compiler/gcc/9.3.0/lib64
2022-07-11 22:07:30.837990: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
%cd /projects/sansar3/DAEN690/tf_object_detection/models/research
!protoc object_detection/protos/*.proto --python_out=.
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .


/projects/sansar3/DAEN690/tf_object_detection/models/research
cp: cannot create regular file './setup.py': Permission denied
Defaulting to user installation because normal site-packages is not writeable
Processing /projects/sansar3/DAEN690/tf_object_detection/models/research
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_py
      copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
      error: could not delete 'build/lib/object_detection/protos/anchor_generator_pb2.py': Permission denied
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for object-detection
  Running setup.py clean for object-detection
Failed to build object-detection
  Running setup.py in

/projects/sansar3/DAEN690/pascals
/bin/bash: unzip: command not found


In [ ]:
%%python - -x /scratch/fpercent/data/train/*/ -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/train_record.tfrecord -i /scratch/fpercent/data/train -c /home/fpercent/hard-headed/Dataset/train_labels.csv
#%%python - -x /scratch/fpercent/data/test/*/ -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/test_record.tfrecord -i /scratch/fpercent/data/test -c /home/fpercent/hard-headed/Dataset/test_labels.csv
#%%python - -x /scratch/fpercent/data/val/*/ -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/val_record.tfrecord -i /scratch/fpercent/data/val -c /home/fpercent/hard-headed/Dataset/val_labels.csv

""" XML-to-TFRecord converter

Arguments [-h] [-x XML_DIR] [-l LABELS_PATH] [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]

optional arguments:
  -h, --help            show this help message and exit
  -x XML_DIR, --xml_dir XML_DIR
                        Path to the folder where the input .xml files are stored.
  -l LABELS_PATH, --labels_path LABELS_PATH
                        Path to the labels (.pbtxt) file.
  -o OUTPUT_PATH, --output_path OUTPUT_PATH
                        Path of output TFRecord (.record) file.
  -i IMAGE_DIR, --image_dir IMAGE_DIR
                        Path to the folder where the input image files are stored. Defaults to the same directory as XML_DIR.
  -c CSV_PATH, --csv_path CSV_PATH
                        Path of output .csv file. If none provided, then no file will be written.
"""

import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
# import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple
print("----------starting -----")
# Initiate argument parser
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str, default="/content/pascals/Annotations")
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str, default="/content/pascals/pascal_label_map.pbtxt")
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str, default="/content/pascals/tfrs")
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default="/content/pascals/JPEGImages")
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

args = parser.parse_args()

if args.image_dir is None:
    args.image_dir = args.xml_dir

label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

print(args.image_dir,"----------------------")
def xml_to_csv(path):
    """Iterates through all .xml files in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """
    print("The path is:", path)
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        print("xml file path:", xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        filename = root.find('filename').text
        print("file name:", filename)
        width = int(root.find('size').find('width').text)
        print("width:", width)
        height = int(root.find('size').find('height').text)
        print("height:", height)
        try:
            for member in root.findall('object'):
                bndbox = member.find('bndbox')
                value = (filename,
                         width,
                         height,
                         member.find('name').text,
                         int(bndbox.find('xmin').text),
                         int(bndbox.find('ymin').text),
                         int(bndbox.find('xmax').text),
                         int(bndbox.find('ymax').text),
                         )
                xml_list.append(value)
        except Exception as ee:
            print(ee)
    print("created the xml list")
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):

    writer = tf.python_io.TFRecordWriter(args.output_path)
    path = os.path.join(args.image_dir)
    examples = xml_to_csv(args.xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))
    if args.csv_path is not None:
        examples.to_csv(args.csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(args.csv_path))


if __name__ == '__main__':
    tf.app.run()

----------starting -----
/scratch/fpercent/data/train ----------------------
The path is: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations
xml file path: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations/hd02.xml
file name: hd02.jpg
width: 4032
height: 3024
xml file path: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations/hd03.xml
file name: hd03.jpg
width: 4032
height: 3024
xml file path: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations/hd04.xml
file name: hd04.jpg
width: 4032
height: 3024
xml file path: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations/hd07.xml
file name: hd07.jpg
width: 4032
height: 3024
xml file path: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations/hd1000.xml
file name: hd1000.jpg
width: 1920
height: 1080
xml file path: /home/fpercent/hard-headed/Dataset/capstone-test/Capstone-test/Annotations/hd1001.xml
file name:

In [ ]:
#%%python - -x /scratch/fpercent/data/train/* -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/train_record.tfrecord -i /scratch/fpercent/data/train -c /home/fpercent/hard-headed/Dataset/train_labels.csv
%%python - -x /scratch/fpercent/data/test/* -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/test_record.tfrecord -i /scratch/fpercent/data/test -c /home/fpercent/hard-headed/Dataset/test_labels.csv
#%%python - -x /scratch/fpercent/data/val/* -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/val_record.tfrecord -i /scratch/fpercent/data/val -c /home/fpercent/hard-headed/Dataset/val_labels.csv

""" XML-to-TFRecord converter

Arguments [-h] [-x XML_DIR] [-l LABELS_PATH] [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]

optional arguments:
  -h, --help            show this help message and exit
  -x XML_DIR, --xml_dir XML_DIR
                        Path to the folder where the input .xml files are stored.
  -l LABELS_PATH, --labels_path LABELS_PATH
                        Path to the labels (.pbtxt) file.
  -o OUTPUT_PATH, --output_path OUTPUT_PATH
                        Path of output TFRecord (.record) file.
  -i IMAGE_DIR, --image_dir IMAGE_DIR
                        Path to the folder where the input image files are stored. Defaults to the same directory as XML_DIR.
  -c CSV_PATH, --csv_path CSV_PATH
                        Path of output .csv file. If none provided, then no file will be written.
"""

import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
# import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple
print("----------starting -----")
# Initiate argument parser
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str, default="/content/pascals/Annotations")
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str, default="/content/pascals/pascal_label_map.pbtxt")
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str, default="/content/pascals/tfrs")
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default="/content/pascals/JPEGImages")
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

args = parser.parse_args()

if args.image_dir is None:
    args.image_dir = args.xml_dir

label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

print(args.image_dir,"----------------------")
def xml_to_csv(path):
    """Iterates through all .xml files in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """
    print("The path is:", path)
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        print("xml file path:", xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        filename = root.find('filename').text
        print("file name:", filename)
        width = int(root.find('size').find('width').text)
        print("width:", width)
        height = int(root.find('size').find('height').text)
        print("height:", height)
        try:
            for member in root.findall('object'):
                bndbox = member.find('bndbox')
                value = (filename,
                         width,
                         height,
                         member.find('name').text,
                         int(bndbox.find('xmin').text),
                         int(bndbox.find('ymin').text),
                         int(bndbox.find('xmax').text),
                         int(bndbox.find('ymax').text),
                         )
                xml_list.append(value)
        except Exception as ee:
            print(ee)
    print("created the xml list")
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):

    writer = tf.python_io.TFRecordWriter(args.output_path)
    path = os.path.join(args.image_dir)
    examples = xml_to_csv(args.xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))
    if args.csv_path is not None:
        examples.to_csv(args.csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(args.csv_path))


if __name__ == '__main__':
    tf.app.run()

In [ ]:
#%%python - -x /scratch/fpercent/data/train/* -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/train_record.tfrecord -i /scratch/fpercent/data/train -c /home/fpercent/hard-headed/Dataset/train_labels.csv
#%%python - -x /scratch/fpercent/data/test/* -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/test_record.tfrecord -i /scratch/fpercent/data/test -c /home/fpercent/hard-headed/Dataset/test_labels.csv
%%python - -x /scratch/fpercent/data/val/* -l /home/fpercent/hard-headed/Dataset/label_map.pbtxt -o /projects/sansar3/DAEN690/hard_headed/val_record.tfrecord -i /scratch/fpercent/data/val -c /home/fpercent/hard-headed/Dataset/val_labels.csv

""" XML-to-TFRecord converter

Arguments [-h] [-x XML_DIR] [-l LABELS_PATH] [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]

optional arguments:
  -h, --help            show this help message and exit
  -x XML_DIR, --xml_dir XML_DIR
                        Path to the folder where the input .xml files are stored.
  -l LABELS_PATH, --labels_path LABELS_PATH
                        Path to the labels (.pbtxt) file.
  -o OUTPUT_PATH, --output_path OUTPUT_PATH
                        Path of output TFRecord (.record) file.
  -i IMAGE_DIR, --image_dir IMAGE_DIR
                        Path to the folder where the input image files are stored. Defaults to the same directory as XML_DIR.
  -c CSV_PATH, --csv_path CSV_PATH
                        Path of output .csv file. If none provided, then no file will be written.
"""

import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
# import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple
print("----------starting -----")
# Initiate argument parser
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str, default="/content/pascals/Annotations")
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str, default="/content/pascals/pascal_label_map.pbtxt")
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str, default="/content/pascals/tfrs")
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default="/content/pascals/JPEGImages")
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

args = parser.parse_args()

if args.image_dir is None:
    args.image_dir = args.xml_dir

label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

print(args.image_dir,"----------------------")
def xml_to_csv(path):
    """Iterates through all .xml files in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """
    print("The path is:", path)
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        print("xml file path:", xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        filename = root.find('filename').text
        print("file name:", filename)
        width = int(root.find('size').find('width').text)
        print("width:", width)
        height = int(root.find('size').find('height').text)
        print("height:", height)
        try:
            for member in root.findall('object'):
                bndbox = member.find('bndbox')
                value = (filename,
                         width,
                         height,
                         member.find('name').text,
                         int(bndbox.find('xmin').text),
                         int(bndbox.find('ymin').text),
                         int(bndbox.find('xmax').text),
                         int(bndbox.find('ymax').text),
                         )
                xml_list.append(value)
        except Exception as ee:
            print(ee)
    print("created the xml list")
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):

    writer = tf.python_io.TFRecordWriter(args.output_path)
    path = os.path.join(args.image_dir)
    examples = xml_to_csv(args.xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))
    if args.csv_path is not None:
        examples.to_csv(args.csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(args.csv_path))


if __name__ == '__main__':
    tf.app.run()